# Convolutional Neural Networks for a Dog Breed Identifier



![Sample Dog Output](images/sample_dog_output.png)

---
<a id='step0'></a>
## Step 0: Import Datasets

Make sure that you've downloaded the required human and dog datasets:
* Download the [dog dataset](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip).  Unzip the folder and place it in this project's home directory, at the location `/dogImages`. 

* Download the [human dataset](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/lfw.zip).  Unzip the folder and place it in the home diretcory, at location `/lfw`.  


---
<a id='step3'></a>
Create a CNN to Classify Dog Breeds (from Scratch)



### (IMPLEMENTATION) Specify Data Loaders for the Dog Dataset

Use the code cell below to write three separate [data loaders](http://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) for the training, validation, and test datasets of dog images (located at `dogImages/train`, `dogImages/valid`, and `dogImages/test`, respectively).  You may find [this documentation on custom datasets](http://pytorch.org/docs/stable/torchvision/datasets.html) to be a useful resource.  If you are interested in augmenting your training and/or validation data, check out the wide variety of [transforms](http://pytorch.org/docs/stable/torchvision/transforms.html?highlight=transform)!

In [1]:
#from sampler import ImbalancedDatasetSampler
import os
from torchvision import datasets
import numpy as np
import torchvision.transforms as transforms
import torch
import torch.nn.functional as F
from torch import nn
import torch.optim as optim
##from torch.utils.data.sampler import Sampler

train_transform = transforms.Compose([
                               transforms.Resize(255),
                               transforms.CenterCrop(224), 
                               transforms.RandomHorizontalFlip(),
                               transforms.RandomRotation(10),
                               transforms.ToTensor(),
                               transforms.Normalize(
                               [0.5,0.5,0.5],
                               [0.5,0.5,0.5])])

test_transform = transforms.Compose([
                               transforms.Resize(255),
                               transforms.CenterCrop(224), 
                               transforms.ToTensor(),
                               transforms.Normalize(
                               [0.5,0.5,0.5],
                               [0.5,0.5,0.5])])

### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes
img_path= (r'D:\deep-learning-pytorch\project-dog-classification\dogImages')

train_data = datasets.ImageFolder(root = img_path +'/train' , transform = train_transform )
valid_data = datasets.ImageFolder(root = img_path +'/valid' , transform = test_transform )
test_data = datasets.ImageFolder(root = img_path +'/test'   , transform = test_transform )

###***train_loader = torch.utils.data.DataLoader(train_data,batch_size = 42,shuffle = True)
train_loader = torch.utils.data.DataLoader(train_data,batch_size = 42,shuffle = True,drop_last=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size = 42,shuffle = True,drop_last=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = 42,shuffle = True,drop_last=True)

loaders= {}
loaders['train'] = train_loader
loaders['valid'] = valid_loader
loaders['test']  = test_loader

loaders_scratch = loaders


### (IMPLEMENTATION) Model Architecture

Create a CNN to classify dog breed.  Use the template in the code cell below.

In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np

use_cuda = torch.cuda.is_available()

# define the CNN architecture
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN
        self.Conv1 =  nn.Conv2d(3,64,3,1)
        self.Conv2 =  nn.Conv2d(64,64,3,1)
        self.pool1 = nn.MaxPool2d(2,2)
        self.Conv3 = nn.Conv2d(64,128,3,1)
        self.Conv4 = nn.Conv2d(128,128,3,1)
        self.pool2 = nn.MaxPool2d(2,2)
        self.Conv5 = nn.Conv2d(128,256,3,1)
        self.Conv6 = nn.Conv2d(256,256,3,1)
        #self.Conv7 = nn.Conv2D(256,256,3,1)
        self.pool3 = nn.MaxPool2d(2,2)
        self.Conv8 = nn.Conv2d(256,512,3,1)
        self.pool4 = nn.MaxPool2d(2,2)
        self.Conv9 = nn.Conv2d(512,512,3,1)
        self.pool5 = nn.MaxPool2d(2,2)
        #self.Conv10 = nn.Conv2d(512,512,3,1)
        self.FC1 =   nn.Linear(512*4*4,1024)
        self.FC2 = nn.Linear(1024,512)
        self.FC3 = nn.Linear(512,133)
        
        ##Dropout of 20%
        
        self.Dropout = nn.Dropout(p=0.2)

        
        
    def forward(self, x):
        ## Define forward behavior
        x = self.Conv1(x)
        x = self.Conv2(x)
        x = F.relu(x) ######
        x = self.pool1(x)
        x = self.Conv3(x)
        x = self.Conv4(x)
        x = F.relu(x) ######
        x = self.pool2(x)
        x = self.Conv5(x)
        x = self.Conv6(x)
        x = F.relu(x) ######
        x = self.pool3(x)
        x = self.Conv8(x)
        x = self.pool4(x)
        x = self.Conv9(x)
        x = self.pool5(x)
        # flatten image input
        #print(x.shape)
        x = x.reshape(-1, 512*4*4)
        x = self.FC1(x) ## added relu
        x = self.Dropout(x) 
        x = self.FC2(x) ## added relu
        x = self.Dropout(x)
        x = self.FC3(x)
        x = F.log_softmax(x,dim=1)
        return x
    
def weights_init_norm(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        n =m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.normal_(0,y)
        m.bias.data.fill_(0)

#-#-#

# instantiate the CNN
model_scratch = Net()

## Added this line
model_scratch.apply(weights_init_norm)

# move tensors to GPU if CUDA is available
if use_cuda:
    model_scratch.cuda()

### Specify Loss Function and Optimizer



In [4]:
import torch.optim as optim
from PIL import ImageFile
import gc
ImageFile.LOAD_TRUNCATED_IMAGES = True

### TODO: select loss function
criterion_scratch = nn.NLLLoss()

### TODO: select optimizer
optimizer_scratch = optim.Adam(model_scratch.parameters(), lr= .0001)

### (IMPLEMENTATION) Train and Validate the Model

Train and validate your model in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at filepath `'model_scratch.pt'`.

In [5]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (image,label) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                image,label = image.cuda(), label.cuda()
            if (batch_idx +1) % 20 == 0:
                print('Batch Id '+ str(batch_idx +1))
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            optimizer_scratch.zero_grad()
            output = model_scratch(image)
            loss = criterion_scratch(output,label)
            ##loss.backward(retain_graph=False)
            loss.backward()
            
            optimizer_scratch.step()
            ##****train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
            ##****torch.cuda.empty_cache() 
            ##print(train_loss)
            #print(loss.item())
            ##image,label = image.to('cpu'), label.to('cpu')
            ##gc.collect()
        ######################    
        # validate the model #
        ######################
        model.eval()
        validation_accuracy = 0.0
        for batch_idx, (image,label) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                image,label = image.cuda(), label.cuda()
            ## update the average validation loss
            
            output = model_scratch(image)
            loss = criterion_scratch(output,label)
            
            prob = torch.exp(output)
            _,class_pred = prob.topk(1, dim =1)
            equals = class_pred== label.view(*class_pred.shape)
            validation_accuracy += torch.mean(equals.type(torch.FloatTensor))            
            
            ##valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        ##Print Validation Accuracy
        print('Validation Accuracy :::::: ' + str(validation_accuracy.item()*100/len(loaders['valid'])) + str(' %'))
        
        ## TODO: save the model if validation loss has decreased
        if (valid_loss < valid_loss_min ):    
            print('Saving Model')
            torch.save(model.state_dict(), 'model_scratch.pt')
            valid_loss_min = valid_loss 
    
    # return trained model
    return model


# train the model
model_scratch = train(18, loaders_scratch, model_scratch, optimizer_scratch, 
                      criterion_scratch, use_cuda, 'model_scratch.pt')

# load the model that got the best validation accuracy
model_scratch.load_state_dict(torch.load('model_scratch.pt'))

Batch Id 20
Batch Id 40
Batch Id 60
Batch Id 80
Batch Id 100
Batch Id 120
Batch Id 140


RuntimeError: CUDA out of memory. Tried to allocate 496.38 MiB (GPU 0; 6.00 GiB total capacity; 3.87 GiB already allocated; 348.64 MiB free; 401.80 MiB cached)

In [6]:
print('current memory allocated: {}'.format(torch.cuda.memory_allocated() / 1024 ** 2))
print('max memory allocated: {}'.format(torch.cuda.max_memory_allocated() / 1024 ** 2))
print('cached memory: {}'.format(torch.cuda.memory_cached() / 1024 ** 2))

current memory allocated: 3457.95166015625
max memory allocated: 3620.21240234375
cached memory: 3700.875
